### This notebook is optionally accelerated with a GPU runtime.
### If you would like to use this acceleration, please select the menu option "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and click "SAVE"

----------------------------------------------------------------------

# GoogLeNet

*Author: Pytorch Team*

**GoogLeNet was based on a deep convolutional neural network architecture codenamed "Inception" which won ImageNet 2014.**



In [ ]:
_ | _
- | -
![alt](https://pytorch.org/assets/images/googlenet1.png) | ![alt](https://pytorch.org/assets/images/googlenet2.png)

### Model Description

GoogLeNet was based on a deep convolutional neural network architecture codenamed "Inception", which was responsible for setting the new state of the art for classification and detection in the ImageNet Large-Scale Visual Recognition Challenge 2014 (ILSVRC 2014). The 1-crop error rates on the ImageNet dataset with a pretrained model are list below.

| Model structure | Top-1 error | Top-5 error |
| --------------- | ----------- | ----------- |
|  googlenet       | 30.22       | 10.47       |



### References

 - [Going Deeper with Convolutions](https://arxiv.org/abs/1409.4842)

Original code from:
https://colab.research.google.com/github/pytorch/pytorch.github.io/blob/master/assets/hub/pytorch_vision_googlenet.ipynb

In [ ]:
import torch
# model = torch.hub.load('pytorch/vision:v0.9.0', 'googlenet', pretrained=True)

import torchvision.models as models
model = models.googlenet(pretrained=True)  #this will not pretrain it

model.eval()

All pre-trained models expect input images normalized in the same way,
i.e. mini-batches of 3-channel RGB images of shape `(3 x H x W)`, where `H` and `W` are expected to be at least `224`.
The images have to be loaded in to a range of `[0, 1]` and then normalized using `mean = [0.485, 0.456, 0.406]`
and `std = [0.229, 0.224, 0.225]`.

Here's a sample execution.

In [ ]:
# Download an example image from the pytorch website
import urllib
url, filename = ("https://github.com/pytorch/hub/raw/master/images/dog.jpg", "dog.jpg")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

In [ ]:
from google.colab import files
files.upload()

In [ ]:
# sample execution (requires torchvision)
from PIL import Image
from torchvision import transforms
# input_image = Image.open(filename)
# input_image = Image.open('car1.jpg')
input_image = Image.open('face5.jpg')
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(input_image)
input_batch = input_tensor.unsqueeze(0) # create a mini-batch as expected by the model

# move the input and model to GPU for speed if available
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)
# Tensor of shape 1000, with confidence scores over Imagenet's 1000 classes
# print(output[0])
# The output has unnormalized scores. To get probabilities, you can run a softmax on it.
probabilities = torch.nn.functional.softmax(output[0], dim=0)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


In [ ]:
# print(probabilities)

In [ ]:
# Download ImageNet labels
!wget https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt

--2021-09-11 15:29:36--  https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10472 (10K) [text/plain]
Saving to: ‘imagenet_classes.txt’

imagenet_classes.tx 100%[===================>]  10.23K  --.-KB/s    in 0s      

2021-09-11 15:29:36 (65.5 MB/s) - ‘imagenet_classes.txt’ saved [10472/10472]



In [ ]:
# Read the categories
with open("imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]
# Show top categories per image
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(categories[top5_catid[i]], top5_prob[i].item())

neck brace 0.15806788206100464
brassiere 0.033983904868364334
toilet seat 0.02665269747376442
hair spray 0.024705681949853897
Band Aid 0.02386784367263317




---

Learn the shape of parts in Inception

https://pytorch.org/vision/stable/_modules/torchvision/models/googlenet.html

In [ ]:
model.conv1.conv  #3 channels, 64 filters

In [ ]:
model.conv1.conv.weight.data.shape  # tensor dims of weight matrix

In [ ]:
len(model.conv1.conv.weight.data)  #num of filters

In [ ]:
model.conv1.conv.weight.data[0].shape  # dims of a filter

In [ ]:
model.conv1.conv.weight.data[0][0].shape  #dims of a filter's channel



---
Modify weights and run again


Create a copy of the model to compare before + after changes for entire Models

In [ ]:
import copy
new_mdl = copy.deepcopy(model)

Set entire weight matrix of a SINGLE FILTER to 0. 

'temp_weightMatrix' is Before

'model.conv1.conv.weight.data[0]' is After

In [ ]:
temp_weightMatrix = model.conv1.conv.weight.data[0].detach().clone()

In [ ]:
new_mdl.conv1.conv.weight.data[0] = new_mdl.conv1.conv.weight.data[0] * 0  

In [ ]:
temp_weightMatrix  #check that copy by value, not by ref

In [ ]:
new_mdl.conv1.conv.weight.data[0] 

In [ ]:
# Run again after tensor modification to see how it changed

new_output = new_mdl(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))



---


Get feature 4c: 447, a car detector. Set its weights to 0. Before and after on car image.

https://distill.pub/2020/circuits/zoom-in/

In [ ]:
model.inception4c.branch3[1].conv.weight[63].shape

torch.Size([24, 3, 3])

In [ ]:
new_mdl = copy.deepcopy(model)
new_mdl.inception4c.branch3[1].conv.weight[63].data = new_mdl.inception4c.branch3[1].conv.weight[63] * 0

In [ ]:
# model.inception4c.branch3[1].conv.weight.data = model.inception4c.branch3[1].conv.weight.data * 0

In [ ]:
new_output = new_mdl(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))

   before:      sports car    0.32742     after:      sports car    0.32742
   before:     convertible    0.18041     after:     convertible    0.18041
   before:         minivan    0.09956     after:         minivan    0.09956
   before:     beach wagon    0.05116     after:     beach wagon    0.05116
   before:          pickup    0.03388     after:          pickup    0.03388


---

Get feature 4b: 418, a dog head pose detector. Set its weights to 0. Before and after on dog image.

https://distill.pub/2020/circuits/zoom-in/

In [ ]:
new_mdl = copy.deepcopy(model)
new_mdl.inception4b.branch3[1].conv.weight[34].data = new_mdl.inception4b.branch3[1].conv.weight[34] * 0

In [ ]:
new_output = new_mdl(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))

   before:      sports car    0.32742     after:      sports car    0.32742
   before:     convertible    0.18041     after:     convertible    0.18041
   before:         minivan    0.09956     after:         minivan    0.09956
   before:     beach wagon    0.05116     after:     beach wagon    0.05116
   before:          pickup    0.03388     after:          pickup    0.03388




---
Try looping thru and modifying layers


In [ ]:
# https://stackoverflow.com/questions/54846905/pytorch-get-all-layers-of-model

named_layers = dict(model.named_modules())
len(named_layers)
named_layers.keys()
# named_layers['conv1']
# type(model.conv1.conv) == torch.nn.modules.conv.Conv2d # True
# type(model.conv1) == torch.nn.modules.conv.Conv2d  # False
# type(named_layers['conv1.conv']) == torch.nn.modules.conv.Conv2d # True

modifiable_layers = []
for layer in list(named_layers.values()):  # layer doesn't use ref; it is a copy of the model's layer
  if type(layer) == torch.nn.modules.conv.Conv2d:
    modifiable_layers.append(layer)

modifiable_layers

In [ ]:
new_mdl = copy.deepcopy(model)
for layer in new_mdl.named_modules():
  if type(layer) == torch.nn.modules.conv.Conv2d:
    layer.weight.data[0] = layer.weight.data[0] * 0

In [ ]:
getattr(new_mdl.conv1, 'conv') = getattr(new_mdl.conv1, 'conv').weight.data



---

Inception seems to be resilient against the change of a filter. Perhaps there are multiple filters that detect dogs. Thus, set entire layers and/or branches to 0.

In [ ]:
new_mdl = copy.deepcopy(model)
# new_mdl.conv1.conv.weight.data = new_mdl.conv1.conv.weight.data * 0
# new_mdl.conv1.bn.weight.data = new_mdl.conv1.bn.weight.data * 0
# new_mdl.conv2.conv.weight.data = new_mdl.conv2.conv.weight.data * 0
new_mdl.conv3.conv.weight.data = new_mdl.conv3.conv.weight.data * 0
# new_mdl.inception3a.branch1.conv.weight.data[0] = new_mdl.inception3a.branch1.conv.weight.data[0] * 0
# new_mdl.inception3a.branch2[0].conv.weight.data[0] = new_mdl.inception3a.branch2[0].conv.weight.data[0] * 0
# new_mdl.inception3a.branch2[1].conv.weight.data[0] = new_mdl.inception3a.branch2[1].conv.weight.data[0] * 0
# new_mdl.inception3a.branch3[0].conv.weight.data[0] = new_mdl.inception3a.branch3[0].conv.weight.data[0] * 0
# new_mdl.inception3a.branch3[1].conv.weight.data[0] = new_mdl.inception3a.branch3[1].conv.weight.data[0] * 0
# new_mdl.inception3b.branch1.conv.weight.data[0] = new_mdl.inception3b.branch1.conv.weight.data[0] * 0

# new_mdl.inception4b.branch3[1].conv.weight.data = new_mdl.inception4b.branch3[1].conv.weight.data * 0
# new_mdl.inception4c.branch3[1].conv.weight.data = new_mdl.inception4c.branch3[1].conv.weight.data * 0

In [ ]:
# new_mdl.inception3a.branch2[0]

In [ ]:
new_output = new_mdl(input_batch)
new_probabilities = torch.nn.functional.softmax(new_output[0], dim=0)
new_top5_prob, new_top5_catid = torch.topk(new_probabilities, 5)
for i in range(new_top5_prob.size(0)):
    row = ['before:', categories[top5_catid[i]], round(top5_prob[i].item(), 5), 'after:', categories[new_top5_catid[i]], round(new_top5_prob[i].item(), 5)]
    print("{: >10} {: >15} {: >10} {: >10} {: >15} {: >10}".format(*row))

   before:         Samoyed    0.93784     after:    space heater     0.4274
   before:      Pomeranian    0.00828     after:     loudspeaker    0.40686
   before:  Great Pyrenees     0.0056     after:   window screen    0.08127
   before:      Arctic fox    0.00553     after:        strainer    0.04828
   before:      white wolf    0.00474     after:           radio    0.00625
